## Setting up notebook

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

#from fairseq.data.data_utils import collate_tokens

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM

from sentence_transformers import SentenceTransformer, util
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup

In [18]:
import re
import sys
import random
from random import choices
import os
from itertools import chain
from string import punctuation
import time
import argparse

import csv
import json

import traceback
import gc
from enum import Enum 

import numpy as np
import pandas as pd
from statistics import mean

import matplotlib.pyplot as plt


# from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
# from trl.ppo import PPOTrainer
#from trl.core import build_bert_batch_from_txt

from IPython.core.display import Markdown,display, HTML, Latex
import qgrid

from verisci.covid import AbstractRetriever, RationaleSelector, LabelPredictor
from verisci.evaluate.lib.data import GoldDataset

from GPUtil import showUtilization as gpu_usage
import wandb

from tqdm.notebook import tqdm
tqdm.pandas()
import pickle

In [19]:
import neptune.new as neptune

nep_run = neptune.init(
    project="ratulalahy/scifact-paraphrase-T5-evo",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2NWQwMGIyZi1mNzM5LTRiMjEtOTg2MC1mNTc4ODRiMWU2ZGYifQ==",
)  # your credentials

# neptune.create_experiment(name = 'org_refute_gen_sup_threshold_0.7', description = 'Trained with org refute gen support , but having \
#                           threshold grater or equal 0.7. ')

https://app.neptune.ai/ratulalahy/scifact-paraphrase-T5-evo/e/SCIF3-117
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [20]:
# import wandb
# wandb.login()
# wandb.init(project="Scifact_paraphrase_T5_per_evo_general_parasci_sup_to_ref_2_ft_0.0.1", entity="qratulalahy")
# from pytorch_lightning.loggers import WandbLogger

# wandb_logger = WandbLogger()

In [21]:
from datetime import datetime
cur_date_time = datetime.today().strftime('%Y_%m_%d_%H_%M')
log_dir = "../../logs/"
project_name = 'Scifact_paraphrase_T5_scifive_per_evo_filter_tech_mmli_v1'
version = '0.0.2'
log_file_dir_name = log_dir+project_name+'_'+version+'.log'

In [22]:
import logging
  
#Create and configure logger
logging.basicConfig(filename=log_file_dir_name,
                    level=logging.INFO,
                    format='%(message)s',
                    filemode='w')

log_file_fine_tune_callback = '../../logs/log_results.txt'

In [23]:
from notify_run import Notify
notify = Notify()
notify.register()

Endpoint: https://notify.run/yNjhNjc4MwGznWXFikuX
To subscribe, open: https://notify.run/c/yNjhNjc4MwGznWXFikuX
Or scan this QR code:

                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              

In [24]:
pd.set_option("display.precision", 2)

In [25]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [26]:
class ParaphraseTargetDirection(Enum):
    org_support_to_gen_refute = 0
    org_refute_to_gen_support = 1
    
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(37)    

In [27]:
PARAPHRASE_PROJECT_SETTINGS={
    'file_and_dirs': {
        'file_paraphrased_no_tune_all_model_full' : '../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl', # can be deleted
        'file_org_claims_by_scifact' : '../../dfs_generated/scifact/org_claim_ext_label_roberta_large_fever.pkl',
    },
    'config_scifact' : {
        'cls_model_name': '/home/qudratealahyratu/research/nlp/fact_checking/my_work/SciMedAttack/ars_joint/ARSJointModel/model/BioBert_large_w.model', #'/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/model/label_roberta_large_fever_scifact',
        'rationale_model_name':'/home/qudratealahyratu/research/nlp/fact_checking/my_work/SciMedAttack/ars_joint/ARSJointModel/model/BioBert_large_w.model', #'/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_fever_scifact',
        'loc_gold_ds_corpus' : '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/data/corpus.jsonl', 
        'loc_gold_ds_train' : '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/data/claims_train.jsonl', 
        'loc_gold_ds_dev' : '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/data/claims_dev.jsonl', 

    },
    
    
    'paraphrase_model' :
    {
        'list_potential_paraphrase_models' : 
            [
                {'model_name' : 'parasci_base_no_fine_tune' , 'model_path_or_url' : 'HelloRusk/t5-base-parasci', 'is_selected' : False},
                {'model_name' : 'parrot_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'parrot_div_base_no_fine_tune' , 'model_path_or_url' : 'prithivida/parrot_paraphraser_on_T5', 'is_selected' : False},
                {'model_name' : 'pegasus_base_no_fine_tune' , 'model_path_or_url' : 'tuner007/pegasus_paraphrase', 'is_selected' : False},
                {'model_name' : 'paws_base_no_fine_tune' , 'model_path_or_url' : 'Vamsi/T5_Paraphrase_Paws', 'is_selected' : True},
                {'model_name' : 'tapaco_base_no_fine_tune' , 'model_path_or_url' : 'hetpandya/t5-base-tapaco', 'is_selected' : False},
                {'model_name' : 'sci_five_pubmed' , 'model_path_or_url' : 'razent/SciFive-large-Pubmed_PMC', 'is_selected' : False}
            ],
        't5_paraphrase_model_params':
        {
            'max_length':256,
            'do_sample':True,
            'top_k':50,
            'top_p': 0.99,
            'repetition_penalty':3.5,
            'early_stopping':True,
            'num_return_sequences':10
        }
    },
    'entailment_model':
    {
        'model_path' : 'pytorch/fairseq',
        'model_name' : 'roberta.large.mnli',
    },
    'labels_multi_nli' :
    {
        0: 'contradiction', 
        1 : 'neutral', 
        2 : 'entailment'
    },
    
    'run_settings':
    {
        'PARAPHRASE_FT_TRAIN_SPLIT' : 0.1,
        'PARAPHRASE_FT_DATASET_DIRECTION' : ParaphraseTargetDirection.org_refute_to_gen_support,#ParaphraseTargetDirection.org_support_to_gen_refute,#ParaphraseTargetDirection.org_refute_to_gen_support,
        'NUM_OF_EPOCH_REQ_FT' : 2,
        'FILTER_BY' : 'TECH_TERMS',
        'SIMILARITY_THRESHOLD' : -100
        #'CUR_MODEL_NAME_PATHS' : (lambda: [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True])(),
    },
}

CUR_NO_OF_EPOCH_FT = 0

In [28]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered ='../../dfs_generated/linguistic/df_scispacy_sentence_word_unq_ner_abr_filtered.pkl'

##  Load paraphrased dataset for selected `paraphrase_models`

In [29]:
def get_paraphrased_dataframe_all_model_no_fine_tuned():
    return pd.read_pickle(PARAPHRASE_PROJECT_SETTINGS['file_and_dirs']['file_paraphrased_no_tune_all_model_full'])

In [30]:
def get_paraphrased_dataframe_selected_models(df_all_model_paraphrased, list_model_names):
    return df_all_model_paraphrased[df_all_model_paraphrased['model'].isin(list_model_names)]

## Prepare dataset for fine tuning

In [31]:
def get_dataframes_by_majority_org_claim(df_all_paraphrased_org_claim):
    df_all_paraphrased_org_success = df_all_paraphrased_org_claim[df_all_paraphrased_org_claim['org_comment'] == 'success']
    
    # Select claims with majority
    df_paraphrased_org_support_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] > df_all_paraphrased_org_success['org_count_refute']
    ]

    
    df_paraphrased_org_refute_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] < df_all_paraphrased_org_success['org_count_refute']
    ]
    
    return df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_org_success


In [32]:
tmp_html_tag = ''

In [33]:
def report_dataframes_by_majority_org_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_org_success):
    count_org_claim_support_major = len(df_paraphrased_org_support_major['org_claim'].unique())
    count_org_claim_refute_major = len(df_paraphrased_org_refute_major['org_claim'].unique())
    count_successful_org_claim = len(df_all_paraphrased_org_success['org_claim'].unique())
    ## Report majority
    tmp_html_tag = ''
    tmp_html_tag += '<h3 style="color:#0080ff">' + 'Original Claim Stat for current SciFact model' +'</h3>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of unique successful claim with Support majority : '+str(count_org_claim_support_major)+'</h4>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of unique successful claim with Refute majority : '+str(count_org_claim_refute_major)+'</h4>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of unique successful claim : '+str(count_successful_org_claim)+'</h4>'
    display(HTML(tmp_html_tag))

In [34]:
def get_df_succesfully_attacked_claim(df_paraphrased_support_major, df_paraphrased_refute_major):
    df_org_refute_gen_support = df_paraphrased_refute_major[
    df_paraphrased_refute_major['gen_count_support'] > df_paraphrased_refute_major['gen_count_refute']
    ]

    df_org_support_gen_refute = df_paraphrased_support_major[
        df_paraphrased_support_major['gen_count_support'] < df_paraphrased_support_major['gen_count_refute']
    ]
    
    return df_org_support_gen_refute, df_org_refute_gen_support



In [35]:
def report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch):
    
    tmp_html_tag = ''
    tmp_html_tag += '<h3 style="color:#0080ff">' + 'Succesfully attacked Claim Stat ' +'</h3>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of toal org refute to gen support : '+str(len(df_org_refute_gen_support))+'</h4>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of total org support to gen refute : '+str(len(df_org_support_gen_refute))+'</h4>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of unique org refute to support : '+str(len(df_org_refute_gen_support['org_claim'].unique()))+'</h4>'
    tmp_html_tag += '<h4 style="color:#0080ff">'+'# of unique org support to refute : '+str(len(df_org_support_gen_refute['org_claim'].unique()))+'</h4>'
    display(HTML(tmp_html_tag))
    
    dict_no_ft_org_gen_count = {
    '# of toal org refute to gen support': len(df_org_refute_gen_support),
    '# of total org support to gen refute' : len(df_org_support_gen_refute), 
    '# of unique org refute to support' : len(df_org_refute_gen_support['org_claim'].unique()),
    '# of unique org support to refute' : len(df_org_support_gen_refute['org_claim'].unique()),
    }

    nep_run['no_ft_org_gen_count'] = dict_no_ft_org_gen_count

In [36]:
'org SUP gen REF'.lower().replace(' ', '_')

'org_sup_gen_ref'

In [37]:
def report_df_filter(df_cur_analyze_filter, name_analyzed_df ,cur_epoch):
    
    name_analyzed_df_formatted = name_analyzed_df.lower().replace(' ', '_')
    num_mlnli_ent_org_gen = len(df_cur_analyze_filter[df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment'])
    num_mlnli_ent_gen_org = len(df_cur_analyze_filter[df_cur_analyze_filter['mlnli_label_gen_org'] == 'entailment'])
    num_mlnli_ent_both = len(df_cur_analyze_filter[
        (df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment') &
        (df_cur_analyze_filter['mlnli_label_gen_org'] == 'entailment')])
    num_passed_ner_abr_filter_ic = len(df_cur_analyze_filter[df_cur_analyze_filter['passed_ner_abr_filter_ic'] == True])
    
    num_both_ent_ner_passed = len(df_cur_analyze_filter[
        (df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment') &
        (df_cur_analyze_filter['mlnli_label_gen_org'] == 'entailment') &
        (df_cur_analyze_filter['passed_ner_abr_filter_ic'] == True)
    ])
    #unique 
    num_unique_mlnli_ent_org_gen = len(df_cur_analyze_filter[df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment']['org_claim'].unique())
    
    num_unique_mlnli_ent_both = len(df_cur_analyze_filter[
        (df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment') &
        (df_cur_analyze_filter['mlnli_label_gen_org'] == 'entailment')]['org_claim'].unique())
    
    num_unique_both_ent_ner_passed = len(df_cur_analyze_filter[
        (df_cur_analyze_filter['mlnli_label_org_gen'] == 'entailment') &
        (df_cur_analyze_filter['mlnli_label_gen_org'] == 'entailment') &
        (df_cur_analyze_filter['passed_ner_abr_filter_ic'] == True)
    ]['org_claim'].unique())
    
    tmp_html_tag = ''
    tmp_html_tag += '<h3 style="color:#004f11">' + 'Filtered for '+name_analyzed_df+' ' +'</h3>'
    tmp_html_tag += '<h4 style="color:#004f11">'+'# mlnli_ent_org_gen : '+str(num_mlnli_ent_org_gen)+'</h4>'
    tmp_html_tag += '<h4 style="color:#004f11">'+'# mlnli_ent_gen_org : '+str(num_mlnli_ent_gen_org)+'</h4>'
    tmp_html_tag += '<h4 style="color:#004f11">'+'# mlnli_ent_both : '+str(num_mlnli_ent_both)+'</h4>'
    tmp_html_tag += '<h4 style="color:#004f11">'+'# passed_ner_abr_filter_ic : '+str(num_passed_ner_abr_filter_ic)+'</h4>'
    tmp_html_tag += '<h4 style="color:#004f11">'+'# both_ent_ner_passed : '+str(num_both_ent_ner_passed)+'</h4>'
    
    tmp_html_tag += '<h4 style="color:#7700a6">'+'# unique_mlnli_ent_org_gen : '+str(num_unique_mlnli_ent_org_gen)+'</h4>'
    tmp_html_tag += '<h4 style="color:#7700a6">'+'# unique_mlnli_ent_both : '+str(num_unique_mlnli_ent_both)+'</h4>'
    tmp_html_tag += '<h4 style="color:#7700a6">'+'# unique_both_ent_ner_passed : '+str(num_unique_both_ent_ner_passed)+'</h4>'
    display(HTML(tmp_html_tag))
    
    dict_no_ft_org_gen_count = {
    name_analyzed_df_formatted+'_mlnli_ent_org_gen': num_mlnli_ent_org_gen,
    name_analyzed_df_formatted+'_mlnli_ent_gen_org' : num_mlnli_ent_gen_org, 
    name_analyzed_df_formatted+'_mlnli_ent_both' : num_mlnli_ent_both,
    name_analyzed_df_formatted+'_passed_ner_abr_filter_ic' : num_passed_ner_abr_filter_ic,
    name_analyzed_df_formatted+'_both_ent_ner_passed' : num_both_ent_ner_passed,
    name_analyzed_df_formatted+'_unique_mlnli_ent_org_gen' : num_unique_mlnli_ent_org_gen,
    name_analyzed_df_formatted+'_unique_mlnli_ent_both' : num_unique_mlnli_ent_both,
    name_analyzed_df_formatted+'_unique_both_ent_ner_passed' : num_unique_both_ent_ner_passed,
    }

    nep_run['no_ft_org_gen_count'] = dict_no_ft_org_gen_count

## Load Abbraviation NER dataframe

In [38]:
df_scispacy_sentence_word_unq_ner_abr_filtered = pd.read_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)

In [39]:
df_scispacy_sentence_word_unq_ner_abr_filtered['ner_text_stripped'] = df_scispacy_sentence_word_unq_ner_abr_filtered['ner_text'].apply(lambda x: re.sub('[^a-z]+', ' ', x.lower()))

In [40]:
df_scispacy_sentence_word_unq_ner_abr_filtered

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source,ner_text_stripped
0,UK,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,18,20,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,uk
1,PrP,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,35,38,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,prp
2,genomes,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",6,13,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genomes
3,genetic sequence,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",41,57,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genetic sequence
4,variants,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",139,147,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,variants
...,...,...,...,...,...,...,...,...,...,...
1724,A20,PROTEIN,en_ner_jnlpba_md,siRNA knockdown of A20 slows tumor progression...,19,22,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train,a
1725,tumor,CANCER,en_ner_bionlp13cg_md,siRNA knockdown of A20 slows tumor progression...,29,34,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train,tumor
1726,murine,TAXON,en_ner_craft_md,siRNA knockdown of A20 slows tumor progression...,61,77,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train,murine
1727,β-sheet,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,0,7,SUPPORTS,[The major conformational changes in PlyB are ...,train,sheet


## Fine Tuning

### Setting up Fine tuner

In [41]:
# train_fine_tune, val_fine_tune = train_test_split(df_paraphrase_fine_tune_dataset[['org_claim', 'gen_claim']], 
#                                                   test_size=paraphraser_split_size)

def get_train_test_dataset(df_to_be_splitted, split_size): 
    df_train, df_val = train_test_split(df_to_be_splitted, 
                                                      test_size=split_size)
    df_train.reset_index(drop=True, inplace=True)
    df_val.reset_index(drop=True, inplace=True)
    return df_train, df_val

In [42]:
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, target_dataframe, max_len=512, truncation=True):
        #self.path = os.path.join(data_dir, type_path + '.csv')

        self.source_column = "org_claim"
        self.target_column = "gen_claim"
        self.data = target_dataframe#pd.read_csv(self.path)
        #print(self.data)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            input_, target = self.data.loc[idx, self.source_column], self.data.loc[idx, self.target_column]

            input_ = "paraphrase: "+ input_ + ' </s>'
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [43]:
def get_dataset(tokenizer, target_dataframe, args):
    return ParaphraseDataset(tokenizer=tokenizer, target_dataframe = target_dataframe,  max_len=args.max_seq_length)

In [44]:
class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams):
        # Calling the super constructer
        super(T5FineTuner,self).__init__()

        self.hparams.update(vars(hparams))

        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):        
        return self.model(input_ids, attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                decoder_attention_mask=decoder_attention_mask,
                labels=labels,)
     
    def is_logger(self):
        return True
        #return self.trainer.proc_rank <= 0        
    
    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100 #########

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss    
    
    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}    
    
    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return None#{"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}    
    
    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}    
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}    
    
    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]    
    
#     def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, using_native_amp=None):
#         if self.trainer.use_tpu:
#             print('why tpu!')
#             xm.optimizer_step(optimizer)
#         else:
#             print('here!')
#             optimizer.step()
#         optimizer.zero_grad()
#         self.lr_scheduler.step()    
     
    def optimizer_step(self, epoch=None, batch_idx=None, optimizer=None, optimizer_idx=None,
                       optimizer_closure=None, on_tpu=None, using_native_amp=None, using_lbfgs=None):
        optimizer.step(closure=optimizer_closure) # remove 'closure=optimizer_closure' here
        optimizer.zero_grad()
        self.lr_scheduler.step()
    
    
    
    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict        
    
    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, target_dataframe=self.hparams.df_train, args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, target_dataframe=self.hparams.df_val, args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [45]:

logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            output_test_results_file = log_file_fine_tune_callback
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [46]:
# Hyper parameters

class FineTuneHyperParams:
    def __init__(self,model_name_path, num_train_epochs, df_train, df_val, df_train_val):
        self.args_dict_fine_tune = dict(
            #data_dir='./tmp_data/', # path for data files
            #output_dir='./tmp_data/', # path to save the checkpoints
            #temp_train_file_name = 'train.csv',
            #temp_validation_file_name = 'val.csv',
            #temp_train_val_file_name = 'all.csv',
            df_train = df_train,
            df_val = df_val,
            df_train_val = df_train_val,
            model_name_or_path= model_name_path,#'HelloRusk/t5-base-parasci',
            tokenizer_name_or_path= model_name_path,#'HelloRusk/t5-base-parasci',
            max_seq_length=512,
            learning_rate=3e-4,
            weight_decay=0.0,
            adam_epsilon=1e-8,
            warmup_steps=0,
            train_batch_size=4,
            eval_batch_size=4,
            num_train_epochs=num_train_epochs,
            gradient_accumulation_steps=16,
            n_gpu=1,
            early_stop_callback=False,
            fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
            opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
            max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
            seed=37,
        )

        self.args_fine_tune_ns = argparse.Namespace(**self.args_dict_fine_tune)

        self.checkpoint_callback_fine_tune = pl.callbacks.ModelCheckpoint(
            dirpath="checkpoints",
            filename="best-checkpoint",
            save_top_k=5,
            verbose=True,
            monitor="val_loss",
            mode="min")

        self.train_params_fine_tune = dict(
            accumulate_grad_batches=self.args_fine_tune_ns.gradient_accumulation_steps,
            gpus=self.args_fine_tune_ns.n_gpu,
            max_epochs=self.args_fine_tune_ns.num_train_epochs,
            #early_stop_callback=False, #
            precision=32,
            #amp_level=self.args_fine_tune_ns.opt_level, #
            gradient_clip_val=self.args_fine_tune_ns.max_grad_norm,
            #logger=wandb_logger,
            callbacks=[self.checkpoint_callback_fine_tune, LoggingCallback()],
        )

## Scifact Functinos

### Load data

In [47]:
def get_claim_label_from_jsonl(dataset_jsonl):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name

            tmp_dic = {"claim" : claim_txt, "label" : claim_label}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [48]:
def get_claim_label_evidence_from_jsonl(dataset_jsonl, source):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = claim_train.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name
            
            list_rationales = []
            for i, sents in enumerate(evidence.rationales):
                list_rationales = [sent for i, sent in enumerate(ev_doc.sentences) if i in sents]

            tmp_dic = {"claim" : claim_txt, "label" : claim_label, "list_rationales" :list_rationales, "source" :source}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [49]:
ds_train = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_train'])
claim_train = ds_train.get_claim(39)
claim_train.pretty_print()

dic_train = get_claim_label_evidence_from_jsonl(ds_train, source = "train")

Example 39: A diminished ovarian reserve does not solely indicate infertility in an a priori non-infertile population.

Evidence sets:

####################

13497630: SUPPORTS
Set 0:
	- After adjusting for age, body mass index, race, current smoking status, and recent hormonal contraceptive use, women with low AMH values (<0.7 ng/mL [n = 84]) did not have a significantly different predicted probability of conceiving by 6 cycles of attempt (65%; 95% CI, 50%-75%) compared with women (n = 579) with normal values (62%; 95% CI, 57%-66%) or by 12 cycles of attempt (84% [95% CI, 70%-91%] vs 75% [95% CI, 70%-79%], respectively).
Set 1:
	- Women with high serum FSH values (>10 mIU/mL [n = 83]) did not have a significantly different predicted probability of conceiving after 6 cycles of attempt (63%; 95% CI, 50%-73%) compared with women (n = 654) with normal values (62%; 95% CI, 57%-66%) or after 12 cycles of attempt (82% [95% CI, 70%-89%] vs 75% [95% CI, 70%-78%], respectively).
Set 2:
	- Women

In [50]:
ds_valid = GoldDataset(PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_corpus'],
                       PARAPHRASE_PROJECT_SETTINGS['config_scifact']['loc_gold_ds_dev'])
claim_valid = ds_valid.get_claim(42)
claim_valid.pretty_print()

dic_valid = get_claim_label_evidence_from_jsonl(ds_valid, source = "dev")

Example 42: A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.

Evidence sets:

####################

18174210: REFUTES
Set 0:
	- Individuals homozygous for alpha(+)-thalassaemia have microcytosis and an increased erythrocyte count.
	- We estimated that the haematological profile in children homozygous for alpha(+)-thalassaemia reduces the risk of SMA during acute malaria compared to children of normal genotype (relative risk 0.52; 95% confidence interval [CI] 0.24-1.12, p = 0.09).   

Set 1:
	- CONCLUSIONS The increased erythrocyte count and microcytosis in children homozygous for alpha(+)-thalassaemia may contribute substantially to their protection against SMA.


In [51]:
df_claim_evid_label = pd.concat([pd.DataFrame(dic_train), pd.DataFrame(dic_valid)], ignore_index=True)

#df_claim_evid_label

In [52]:
df_claim_evid_label

,claim,label,list_rationales,source
0,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train
1,32% of liver transplantation programs required...,SUPPORTS,[Policies requiring discontinuation of methado...,train
2,40mg/day dosage of folic acid and 2mg/day dosa...,SUPPORTS,[CONCLUSION Treatment with high doses of folic...,train
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train
4,A T helper 2 cell (Th2) environment impedes di...,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train
...,...,...,...,...
768,Women with a higher birth weight are more like...,SUPPORTS,[Increased risk of breast cancer was noted wit...,dev
769,Women with a higher birth weight are more like...,SUPPORTS,[RESULTS We found that heavier birth weights w...,dev
770,aPKCz causes tumour enhancement by affecting g...,REFUTES,"[Taken together, this demonstrates that PKCζ i...",dev
771,cSMAC formation enhances weak ligand signalling.,SUPPORTS,[This conclusion was supported by experiments ...,dev


### Scifact Model

In [53]:
class ArgsScifact:
    def __init__(self, claim):
        self.claim = claim
        self.report_file = "/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/results/covid/report" #not needed
        self.n_documents = 100
        self.rationale_selection_method = "topk"
        self.output_format = "markdown"
        self.rationale_threshold = 0.5
        self.label_threshold = 0.5
        self.keep_nei = False
        self.full_abstract = True
        self.verbose = True
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        ##
class PretrainedModelsForScifact:
    def __init__(self, args):
        if args.device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(args.device)
            
        #self.rationale_selection_model = '/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/model/rationale_roberta_large_scifact'
        self.rationale_selection_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['rationale_model_name']
        self.label_prediction_model = PARAPHRASE_PROJECT_SETTINGS['config_scifact']['cls_model_name']
        self.abstract_retriever = AbstractRetriever()
        self.rationale_selector = RationaleSelector(self.rationale_selection_model,
                                               args.rationale_selection_method,
                                               args.rationale_threshold,
                                               self.device)
        self.label_predictor = LabelPredictor(self.label_prediction_model,
                                         args.keep_nei,
                                         args.label_threshold,
                                         self.device)


In [54]:
args_sci = ArgsScifact("")

pretrained_models_config = PretrainedModelsForScifact(args_sci)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 64: invalid start byte

In [ ]:
log_failed_claim = []
def inference(args, pretraind_models_config):

    try:
#         if args.verbose:
#         print("Retrieving abstracts. inference > ", args.claim)
        results = pretraind_models_config.abstract_retriever(args.claim, k=args.n_documents)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'abstract retrival', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        #print("abstract_retriever >> ", results)

#         if args.verbose:
#             print("Selecting rationales. inference > ", args.claim)
        results = pretraind_models_config.rationale_selector(args.claim, results)
        if len(results) == 0:
            temp_dic = {'failed_in' : 'Rationale selection', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
#         if args.verbose:
#             print("Label predictions. inference > ", args.claim)
        results = pretraind_models_config.label_predictor(args.claim, results)

        if len(results) == 0:
            temp_dic = {'failed_in' : 'Label Prediction', 'claim': args.claim}
            log_failed_claim.append(temp_dic)
            return []
        
        results.sort(key=lambda r: r['label_confidence'], reverse=True)
        return results
    except Exception as e:
        print("Exception :: Inference cant retrive info for >> ", args.claim)
        print(sys.exc_info()[0])
        print(traceback.format_exc())
        temp_dic = {'failed_in' : sys.exc_info()[0], 'claim': args.claim}
        log_failed_claim.append(temp_dic)
        return []


In [ ]:
def write_result(result, full_abstract):
    all_msg = ""
    all_msg = f"#### [{result['title']}]({result['url']}) \n"
    #print(msg, file=f)
    #all_msg = all_msg+msg
    ev_scores = [f"{x:0.2f}" for x in result["evidence_confidence"]]
    ev_scores = ", ".join(ev_scores)
    if result['label'].lower() == "support":
        msg = f"🟩 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    elif result['label'].lower() == "refute":
        msg = f"🟥 **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    else:
        msg = f"⏺ **Decision** : {result['label']} (score={result['label_confidence']:0.2f}, evidence scores={ev_scores})\n"
    #print(msg, file=f)
    all_msg = all_msg+msg 
    
    for i, line in enumerate(result["abstract"]):
        # If we're showing the full abstract, show evidence in green.
        if full_abstract:
            if result['label'].lower() == "support":
                msg = (f"- <span style='color:green'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")
            elif result['label'].lower() == "refute":
                msg = (f"- <span style='color:red'>{line}</span>"
                       if i in result["evidence"]
                       else f"- {line}")                
            #print(msg, file=f)
            all_msg = all_msg+msg + " \n"
        else:
            if i in result["evidence"]:
                msg = f"- {line}"
                #print(msg, file=f)
                all_msg = all_msg+msg + " \n" 
    
    #print(file=f)
    #print(40 * "-", file=f)
    #print(file=f)
    all_msg = all_msg+msg 
    return all_msg + "\n"

In [ ]:
def export(args, results):
    all_msg = ""
    claim = args.claim
    #report_file = args.report_file
    #f = open(f"{report_file}.md", "w")
    msg = f"### Claim \n > **{claim}** \n "
    #print(msg, file=f)
    #print(file=f)
    all_msg = all_msg +msg
    
    #support_confs = [], refute_confs = []
    confs = []
    for result in results:
        if result['label'].lower() == "support":
            tmp_dic = {'label' : 'Support', 'label_confidence' : result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        elif result['label'].lower() == "refute":
            tmp_dic = {'label' : 'Refute', 'label_confidence' : -result["label_confidence"], "no_of_evidence" : len(result['evidence_confidence'])}
            confs.append(tmp_dic)
        
    
    tpm_df = pd.DataFrame(confs)
    #HTML(tpm_df.style.bar(align='mid', color=['#d65f5f', '#5fba7d']))
    display(HTML(tpm_df.style.bar(subset=["label_confidence"], align='mid', color=['#ffa1a1', '#bfffcf']).render()))
    
    msg = "### Evidence \n "
    all_msg = all_msg +msg
    for result in results:
        cur_msg = write_result(result, args.full_abstract)
        all_msg = all_msg +cur_msg+"\n"

    return all_msg

In [ ]:
claim_to_check = "Incidence of sepsis fell substantially between the years 2009 and 2014."
args_sci = ArgsScifact(claim_to_check)

#pretrained_models_config = pretrained_models_for_scifact(args_sci)

results_raw = inference(args_sci, pretrained_models_config)

if results_raw!= []:
    result_md = export(args_sci, results_raw)
    #result_md = export(args_sci, results_raw)
    display(Markdown(result_md))

In [55]:
results_raw

NameError: name 'results_raw' is not defined

Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Experiencing connection interruptions. Will try to reestablish communication with Neptune. Internal exception was: RequestsFutureAdapterConnectionError
Communication with Neptune restored!
Communication with Neptune restored!


In [43]:
#print(pretraind_models_config)

## Filter

### Tech term

In [44]:
#https://stackoverflow.com/questions/29996079/match-a-whole-word-in-a-string-using-dynamic-regex
def filter_and_replace_tech_term_paraphrased_claim(claim_paraphrased, claim_original):
    #claim_para_trimmed = re.sub('[^a-z]+', ' ', claim_paraphrased.lower())
    df_cur_sentence_word_unq_ner_abr_filtered = df_scispacy_sentence_word_unq_ner_abr_filtered[
        df_scispacy_sentence_word_unq_ner_abr_filtered['claim'] == claim_original
    ]
    for cur_term_row in df_cur_sentence_word_unq_ner_abr_filtered.itertuples(index=False):
        cur_term_row_formatted = r'(?<!\S){}(?!\S)'.format(re.escape(cur_term_row.ner_text))
        res_num = re.findall(cur_term_row_formatted, claim_paraphrased)
        if res_num == []:
            return False
        
    return True

### Entailment

In [45]:
model_neg_checker_roberta = torch.hub.load(PARAPHRASE_PROJECT_SETTINGS['entailment_model']['model_path'], 
                                           PARAPHRASE_PROJECT_SETTINGS['entailment_model']['model_name'])

model_neg_checker_roberta.to(device)
#model_neg_checker_roberta.cuda()

model_neg_checker_roberta.eval() 

Using cache found in /home/qudratealahyratu/.cache/torch/hub/pytorch_fairseq_main


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [46]:
def get_mlnli_label(org_claim, gen_claim):    
    tokens_sentences_org_gen = model_neg_checker_roberta.encode(org_claim, gen_claim)
    logprobs_sentences_org_gen = model_neg_checker_roberta.predict('mnli', tokens_sentences_org_gen)      
    cal_val_mlnli_org_gen = logprobs_sentences_org_gen.argmax(dim=1).item()
    cal_label_mlnli_org_gen = PARAPHRASE_PROJECT_SETTINGS['labels_multi_nli'][cal_val_mlnli_org_gen]
    
    tokens_sentences_gen_org = model_neg_checker_roberta.encode(gen_claim, org_claim)
    logprobs_sentences_gen_org = model_neg_checker_roberta.predict('mnli', tokens_sentences_gen_org)      
    cal_val_mlnli_gen_org = logprobs_sentences_gen_org.argmax(dim=1).item()
    cal_label_mlnli_gen_org = PARAPHRASE_PROJECT_SETTINGS['labels_multi_nli'][cal_val_mlnli_gen_org]    
#     return {'val_mlnli_org_gen' : cal_val_mlnli_org_gen, 
#             'label_mlnli_org_gen': cal_label_mlnli_org_gen, 
#             'val_mlnli_gen_org': cal_val_mlnli_gen_org, 
#             'label_mlnli_gen_org': cal_label_mlnli_gen_org}
    
    return pd.Series([cal_val_mlnli_org_gen, cal_label_mlnli_org_gen, cal_val_mlnli_gen_org, cal_label_mlnli_gen_org])

## Apply Finetuned Model

In [47]:
def get_t5_gen_sentences(org_sentence, model_t5, tokenizer_t5):
    text =  "paraphrase: " + org_sentence + " </s>"

    encoding = tokenizer_t5.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    #PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']
    outputs = []
    if type(model_t5) == T5ForConditionalGeneration:
        outputs = model_t5.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['max_length'],
            do_sample=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['do_sample'],
            top_k=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_k'],
            top_p=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_p'],
            repetition_penalty=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['repetition_penalty'],
            early_stopping=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['early_stopping'],
            num_return_sequences=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['num_return_sequences']
        )

    else:
        outputs = model_t5.model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['max_length'],
            do_sample=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['do_sample'],
            top_k=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_k'],
            top_p=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_p'],
            repetition_penalty=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['repetition_penalty'],
            early_stopping=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['early_stopping'],
            num_return_sequences=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['num_return_sequences']
        )
        
    gen_sentences_t5 = []
    for output in outputs:
        line = tokenizer_t5.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        gen_sentences_t5.append(line)
    
    #print('gen_sentences_t5_tapaco >> ', gen_sentences_t5_tapaco)
    return list(set(gen_sentences_t5))#[:5]

In [48]:
def get_stat_of_original_claim(row_org_claim):
    claim =  row_org_claim["org_claim"]
    logging.info("#### \n\n>>> Original claim >>> ")
    logging.info(claim)

    args_sci = ArgsScifact(claim)
    dic_info = {}
    
    dic_info["org_claim"] = row_org_claim["org_claim"]
    dic_info["ground_label"] = row_org_claim["ground_label"]
    dic_info["ground_list_rationales"] = row_org_claim["ground_list_rationales"]
    dic_info["source"] = row_org_claim["source"]
    dic_info["org_count_support"] = 0
    dic_info["org_count_refute"] = 0
    dic_info["org_list_supported_ids"] = []
    dic_info["org_list_refuted_ids"] = []
    dic_info["org_list_supported_confidence"] = []
    dic_info["org_list_refuted_confidence"] = []
    dic_info["org_list_supported_confidence_mean"] = 0.0
    dic_info["org_list_refuted_confidence_mean"] = 0.0
    dic_info["org_comment"] = ""
    
    try:
        results_raw_org = inference(args_sci, pretrained_models_config)  
        
        if results_raw_org == []:
            dic_info["org_comment"] = "no result"
            
        else:
            list_supported_ids = [cur_result['id'] for cur_result in results_raw_org if cur_result['label'] == 'SUPPORT']
            list_refuted_ids = [cur_result['id'] for cur_result in results_raw_org if cur_result['label'] == 'REFUTE']
            list_supported_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_org if cur_result['label'] == 'SUPPORT']
            list_refuted_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_org if cur_result['label'] == 'REFUTE']
            
            
            dic_info["org_count_support"] = len(list_supported_ids)
            dic_info["org_count_refute"] = len(list_refuted_ids)
            dic_info["org_list_supported_ids"] = list_supported_ids
            dic_info["org_list_refuted_ids"] = list_refuted_ids
            dic_info["org_list_supported_confidence"] = list_supported_label_confidence
            dic_info["org_list_refuted_confidence"] = list_refuted_label_confidence
            if len(list_supported_label_confidence) > 0:
                dic_info["org_list_supported_confidence_mean"] = mean(list_supported_label_confidence)
            if len(list_refuted_label_confidence) > 0:
                dic_info["org_list_refuted_confidence_mean"] = mean(list_refuted_label_confidence)
            dic_info["org_comment"] = "success"
            
    except Exception as e:
        dic_info["org_comment"] = "exception : "+e
        logging.info(">>> Exception original claim >>> ") 
        logging.info(claim) 
        logging.info(e)
    
    finally:
        return dic_info

In [49]:
def get_results_by_gen_claim(gen_claim, dic_original_claim_info):
    logging.info(":: Generated claim :: ")
    logging.info(gen_claim)
    #print(gen_claim)
    
    args_gen = ArgsScifact(gen_claim)
    gen_dic_info = {}
    
    gen_dic_info["gen_claim"] = gen_claim
    gen_dic_info["gen_count_support"] = 0
    gen_dic_info["gen_count_refute"] = 0
    gen_dic_info["gen_list_supported_ids"] = []
    gen_dic_info["gen_list_refuted_ids"] = []
    gen_dic_info["gen_list_supported_confidence"] = []
    gen_dic_info["gen_list_refuted_confidence"] = []
    gen_dic_info["gen_list_supported_confidence_mean"] = 0.0
    gen_dic_info["gen_list_refuted_confidence_mean"] = 0.0
    gen_dic_info["gen_comment"] = ""    
    
    gen_dic_info["common_all"] = 0
    gen_dic_info["common_support_refute"] = 0
    gen_dic_info["common_refute_support"] = 0
    gen_dic_info["common_support_support"] = 0
    gen_dic_info["common_refute_refute"] = 0
    
    try:
        results_raw_gen = inference(args_gen, pretrained_models_config)  

        if results_raw_gen == []:
            gen_dic_info["gen_comment"] = "no result"
            
        else:
            list_supported_ids = [cur_result['id'] for cur_result in results_raw_gen if cur_result['label'] == 'SUPPORT']
            list_refuted_ids = [cur_result['id'] for cur_result in results_raw_gen if cur_result['label'] == 'REFUTE']
            list_supported_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_gen if cur_result['label'] == 'SUPPORT']
            list_refuted_label_confidence =  [cur_result['label_confidence'] for cur_result in results_raw_gen if cur_result['label'] == 'REFUTE']
            
            
            gen_dic_info["gen_count_support"] = len(list_supported_ids)
            gen_dic_info["gen_count_refute"] = len(list_refuted_ids)
            gen_dic_info["gen_list_supported_ids"] = list_supported_ids
            gen_dic_info["gen_list_refuted_ids"] = list_refuted_ids
            gen_dic_info["gen_list_supported_confidence"] = list_supported_label_confidence
            gen_dic_info["gen_list_refuted_confidence"] = list_refuted_label_confidence
            if len(list_supported_label_confidence) > 0 :
                gen_dic_info["gen_list_supported_confidence_mean"] = mean(list_supported_label_confidence)
            if len(list_refuted_label_confidence) > 0:
                gen_dic_info["gen_list_refuted_confidence_mean"] = mean(list_refuted_label_confidence)
            gen_dic_info["gen_comment"] = "success"      
            
            
            common_all = (set(gen_dic_info["gen_list_supported_ids"]) | set(gen_dic_info["gen_list_refuted_ids"])) & \
                (set(dic_original_claim_info["org_list_supported_ids"]) | set(dic_original_claim_info["org_list_refuted_ids"]))
            
            common_support_refute = set(dic_original_claim_info["org_list_supported_ids"]) & set(gen_dic_info["gen_list_refuted_ids"])
            common_refute_support = set(dic_original_claim_info["org_list_refuted_ids"]) & set(gen_dic_info["gen_list_supported_ids"])
            common_support_support = set(dic_original_claim_info["org_list_supported_ids"]) & set(gen_dic_info["gen_list_supported_ids"])
            common_refute_refute = set(dic_original_claim_info["org_list_refuted_ids"]) & set(gen_dic_info["gen_list_refuted_ids"])
            
            gen_dic_info["common_all"] = len(common_all)
            gen_dic_info["common_support_refute"] = len(common_support_refute)
            gen_dic_info["common_refute_support"] = len(common_refute_support)
            gen_dic_info["common_support_support"] = len(common_support_support)
            gen_dic_info["common_refute_refute"] = len(common_refute_refute)
            
            gen_dic_info["gen_comment"] = "success" 
            
    except Exception as e:
        dic_info["gen_comment"] = "exception : "+e
        logging.info(">>> Exception gen claim >>> ") 
        logging.info(claim) 
        logging.info(e)
        
    finally:
        return gen_dic_info           

In [50]:
def get_paraphrased_sentence_no_ft_with_detail_stat(df_dataset_to_be_paraphrased, model_t5, tokenizer_t5, model_name_t5):
    '''
    args:
    df_dataset_to_be_paraphrased : Dataset, those were successfully retrived by current scifact model
    '''
    list_results_fine_tuned = []
    dic_key_sentence_info = model_name_t5+"_sentences_info"
    
    for index_df, cur_row in tqdm(df_dataset_to_be_paraphrased.iloc[:,:].iterrows(), total=len(df_dataset_to_be_paraphrased)):
        cur_res = {}

        dic_info_org_claim = get_stat_of_original_claim(cur_row)
        cur_res["org_claim_info"] = dic_info_org_claim    
        #print(cur_res)
        
        try:
            list_paraphrased_claims = get_t5_gen_sentences(org_sentence = cur_row["org_claim"], 
                                                          model_t5 = model_t5, tokenizer_t5 = tokenizer_t5)#get_t5_gen_sentences(cur_row["org_claim"])
            
            list_paraphrased_claims_with_sim_threshold = []
            for cur_paraphrased_sent in list_paraphrased_claims:
                ## Enable if need to measure similarity score                
                #                 cur_similarity_score = get_sentence_similarity_score(model_sim_diltillroberta_base,
                #                                                                     cur_row['org_claim'], 
                #                                                                     cur_paraphrased_sent)
                cur_similarity_score = 1.0
                if cur_similarity_score >= PARAPHRASE_PROJECT_SETTINGS['run_settings']['SIMILARITY_THRESHOLD']:
                    list_paraphrased_claims_with_sim_threshold.append(cur_paraphrased_sent)
            #Filter paraphrased sentences with tech terms
                
            #for cur_paraphrased_sent in list_paraphrased_claims_with_sim_threshold:
                
            
            list_dic_paraphrased_info = []
            for cur_paraphrased_sent in list_paraphrased_claims_with_sim_threshold:
                cur_dic_paraphraased_claim_info = get_results_by_gen_claim(cur_paraphrased_sent, dic_info_org_claim)
                cur_dic_paraphraased_claim_info["model"] = model_name_t5
                
#                 cur_dic_paraphraased_claim_info['passed_ner_abr_filter_ic'] = filter_and_replace_tech_term_paraphrased_claim(cur_paraphrased_sent, 
#                                                                                                                              dic_info_org_claim['org_claim'])
#                 dict_mlnli_labels = get_mlnli_label(dic_info_org_claim['org_claim'], cur_paraphrased_sent)
#                 cur_dic_paraphraased_claim_info.update(dict_mlnli_labels)
                
                list_dic_paraphrased_info.append(cur_dic_paraphraased_claim_info)                 
                

            cur_res[dic_key_sentence_info] = list_dic_paraphrased_info
        except Exception as e:
            logging.info(">>> Exception genereted claim >>> ")
            logging.info(cur_row["org_claim"])
            logging.info(e)     
            print('exc : ', e)
        #print(cur_res)
        list_results_fine_tuned.append(cur_res)
            
    #print(len(list_results_fine_tuned))
    ## Formatting dataframe
    result_as_dict = []
    for cur_claim in list_results_fine_tuned:
        #print(cur_claim.keys())
        for cur_gen_paraphrased_claim in cur_claim[dic_key_sentence_info]:
            cur_merged_dict = {**cur_claim["org_claim_info"], **cur_gen_paraphrased_claim}
            result_as_dict.append(cur_merged_dict)
            #print('cur_merged_dict : ', cur_merged_dict)
    #print(len(result_as_dict))
    return pd.DataFrame(result_as_dict)

In [51]:
def get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased, model_t5, tokenizer_t5, model_name_t5):
    '''
    args:
    df_dataset_to_be_paraphrased : Dataset, those were successfully retrived by current scifact model
    '''
    list_results_fine_tuned = []
    dic_key_sentence_info = model_name_t5+"_sentences_info"
    
    for index_df, cur_row in tqdm(df_dataset_to_be_paraphrased.iloc[:,:].iterrows(), total=len(df_dataset_to_be_paraphrased)):
        cur_res = {}

        dic_info_org_claim = get_stat_of_original_claim(cur_row)
        cur_res["org_claim_info"] = dic_info_org_claim    
        #print(cur_res)
        
        try:
            list_paraphrased_claims = get_t5_gen_sentences(org_sentence = cur_row["org_claim"], 
                                                          model_t5 = model_t5, tokenizer_t5 = tokenizer_t5)#get_t5_gen_sentences(cur_row["org_claim"])
            
            list_paraphrased_claims_with_sim_threshold = []
            for cur_paraphrased_sent in list_paraphrased_claims:
                ## Enable if need to measure similarity score                
                #                 cur_similarity_score = get_sentence_similarity_score(model_sim_diltillroberta_base,
                #                                                                     cur_row['org_claim'], 
                #                                                                     cur_paraphrased_sent)
                cur_similarity_score = 1.0
                if cur_similarity_score >= PARAPHRASE_PROJECT_SETTINGS['run_settings']['SIMILARITY_THRESHOLD']:
                    list_paraphrased_claims_with_sim_threshold.append(cur_paraphrased_sent)
            #Filter paraphrased sentences with tech terms
                
            #for cur_paraphrased_sent in list_paraphrased_claims_with_sim_threshold:
                
            
            list_dic_paraphrased_info = []
            for cur_paraphrased_sent in list_paraphrased_claims_with_sim_threshold:
                cur_dic_paraphraased_claim_info = get_results_by_gen_claim(cur_paraphrased_sent, dic_info_org_claim)
                cur_dic_paraphraased_claim_info["model"] = model_name_t5
                
#                 cur_dic_paraphraased_claim_info['passed_ner_abr_filter_ic'] = filter_and_replace_tech_term_paraphrased_claim(cur_paraphrased_sent, 
#                                                                                                                              dic_info_org_claim['org_claim'])
#                 dict_mlnli_labels = get_mlnli_label(dic_info_org_claim['org_claim'], cur_paraphrased_sent)
#                 cur_dic_paraphraased_claim_info.update(dict_mlnli_labels)
                
                list_dic_paraphrased_info.append(cur_dic_paraphraased_claim_info)                 
                
            cur_res[dic_key_sentence_info] = list_dic_paraphrased_info      
            
        except Exception as e:
            logging.info(">>> Exception genereted claim >>> ")
            logging.info(cur_row["org_claim"])
            logging.info(e)     
            print('exc : ', e)
        #print(cur_res)
        list_results_fine_tuned.append(cur_res)
            
    #print(len(list_results_fine_tuned))
    ## Formatting dataframe
    result_as_dict = []
    for cur_claim in list_results_fine_tuned:
        #print(cur_claim.keys())
        for cur_gen_paraphrased_claim in cur_claim[dic_key_sentence_info]:
            cur_merged_dict = {**cur_claim["org_claim_info"], **cur_gen_paraphrased_claim}
            result_as_dict.append(cur_merged_dict)
            #print('cur_merged_dict : ', cur_merged_dict)
    #print(len(result_as_dict))
    return pd.DataFrame(result_as_dict)

In [52]:
df_org_claims_by_scifact = pd.read_pickle(PARAPHRASE_PROJECT_SETTINGS['file_and_dirs']['file_org_claims_by_scifact'])

In [ ]:
# Load experiment setup
paraphrase_model_path_url = [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_path_url = paraphrase_model_path_url[0]
list_paraphrase_model_names = [_x['model_name'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_name = list_paraphrase_model_names[0]


# Load no-tuned scifact model
model_t5_not_fine_tuned = AutoModelForSeq2SeqLM.from_pretrained(paraphrase_model_path_url)
tokenizer_t5_not_fine_tuned = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
_ = model_t5_not_fine_tuned.to(device)


#remove duplicates from the original dataset
df_org_claims_by_scifact = df_org_claims_by_scifact.drop_duplicates('org_claim', keep='first')
#get paraphrased sentences from no-tuned model
df_paraphrased_selected_model_full = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_org_claims_by_scifact, 
                                                                               model_t5 = model_t5_not_fine_tuned, 
                                                                               tokenizer_t5 = tokenizer_t5_not_fine_tuned, 
                                                                               model_name_t5 = paraphrase_model_name)

model_t5_not_fine_tuned = model_t5_not_fine_tuned.cpu()    
del model_t5_not_fine_tuned

## Pass fillter

# Filter for entailment check
df_paraphrased_selected_model_full[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_selected_model_full.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

#Filter for tech terms check
df_paraphrased_selected_model_full['passed_ner_abr_filter_ic'] = df_paraphrased_selected_model_full.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)

# pass all valids to scifact

df_paraphrased_filtered = df_paraphrased_selected_model_full[
    (df_paraphrased_selected_model_full['passed_ner_abr_filter_ic'] == True) &
    (df_paraphrased_selected_model_full['mlnli_label_org_gen'] == 'entailment') &
    (df_paraphrased_selected_model_full['mlnli_label_gen_org'] == 'entailment')
    
]
#Check majority
df_org_support_major, df_org_refute_major, df_all_cur_model_filtered = get_dataframes_by_majority_org_claim(df_paraphrased_filtered)
report_dataframes_by_majority_org_claim(df_org_support_major, df_org_refute_major, df_all_cur_model_filtered)

# Check attacks by scifact
df_org_support_gen_refute, df_org_refute_gen_support = get_df_succesfully_attacked_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major)
df_org_support_gen_refute['attack_type'] = 'org_sup_to_gen_ref'
df_org_refute_gen_support['attack_type'] = 'org_ref_to_gen_sup'
report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch = CUR_NO_OF_EPOCH_FT)

df_successful_filter_attacked = pd.concat([df_org_support_gen_refute, df_org_refute_gen_support], ignore_index=True)
fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'

with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
    pickle.dump(df_successful_filter_attacked, fp)
    
while(True):
    
# Fine tune model
    df_fine_tuning_dataset = None

                                        
    ## Select dataset for fine-tuning ::either support major or refute major
    if PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_support_to_gen_refute:
        df_fine_tuning_dataset = df_successful_filter_attacked[df_successful_filter_attacked['attack_type'] == 'org_sup_to_gen_ref']
    elif PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_refute_to_gen_support:
        df_fine_tuning_dataset = df_successful_filter_attacked[df_successful_filter_attacked['attack_type'] == 'org_refute_gen_sup']
    else:
        raise ValueError('Select a direction of fine tuning dataset')
        
    CUR_NO_OF_EPOCH_FT += 1
                                        
                                        

    ## Train model with fine-tuning dataset
    df_fine_tuning_dataset.reset_index(drop=True, inplace=True)
    train_split_size = PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_TRAIN_SPLIT']
    df_train_fine_tune, df_validate_fine_tune = get_train_test_dataset(df_fine_tuning_dataset, train_split_size)

    num_train_epochs = PARAPHRASE_PROJECT_SETTINGS['run_settings']['NUM_OF_EPOCH_REQ_FT']
    fineTuneHyperParam = FineTuneHyperParams(model_name_path = paraphrase_model_path_url, 
                                             num_train_epochs = num_train_epochs, df_train = df_train_fine_tune, 
                                             df_val = df_validate_fine_tune, df_train_val = df_fine_tuning_dataset)

    model_t5_fine_tuned = T5FineTuner(fineTuneHyperParam.args_fine_tune_ns)
    trainer_model_t5_fine_tune = pl.Trainer(**fineTuneHyperParam.train_params_fine_tune)
    trainer_model_t5_fine_tune.fit(model_t5_fine_tuned)        
    
                                        
    # ask model to generate paraphrase
    tokenizer_t5 = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
    _ = trainer_model_t5_fine_tune.model.to(device)
    ## Ask fine-tuned model to paraphrase

    ## Evaluate paraphrased dataset
    df_paraphrased_fine_tuned = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_org_claims_by_scifact.iloc[:, :], 
                                              model_t5 = model_t5_fine_tuned, 
                                              tokenizer_t5 = tokenizer_t5, 
                                              model_name_t5 = paraphrase_model_name)

    # Apply filter
    # Filter for entailment check
    df_paraphrased_fine_tuned[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_fine_tuned.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

    #Filter for tech terms check
    df_paraphrased_fine_tuned['passed_ner_abr_filter_ic'] = df_paraphrased_fine_tuned.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)

    # pass all valids to scifact

    df_paraphrased_filtered_fine_tuned = df_paraphrased_fine_tuned[
        (df_paraphrased_fine_tuned['passed_ner_abr_filter_ic'] == True) &
        (df_paraphrased_fine_tuned['mlnli_label_org_gen'] == 'entailment') &
        (df_paraphrased_fine_tuned['mlnli_label_gen_org'] == 'entailment')

    ]
    #Check majority
    df_org_support_major, df_org_refute_major, df_all_cur_model_filtered = get_dataframes_by_majority_org_claim(df_paraphrased_filtered)
    report_dataframes_by_majority_org_claim(df_org_support_major, df_org_refute_major, df_all_cur_model_filtered)

    # Check attacks by scifact
    df_org_support_gen_refute, df_org_refute_gen_support = get_df_succesfully_attacked_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major)
    df_org_support_gen_refute['attack_type'] = 'org_sup_to_gen_ref'
    df_org_refute_gen_support['attack_type'] = 'org_ref_to_gen_sup'
    report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch = CUR_NO_OF_EPOCH_FT)

    df_successful_filter_attacked = pd.concat([df_org_support_gen_refute, df_org_refute_gen_support], ignore_index=True)
    fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'

    with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
        pickle.dump(df_successful_filter_attacked, fp)


  0%|          | 0/692 [00:00<?, ?it/s]

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# get all model's no_fine_tuned dataset
#df_paraphrased_all_model_full = get_paraphrased_dataframe_all_model_no_fine_tuned()

#Filter and select dataset only for the selected model

'''
paraphrase_model_path_url = [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_path_url = paraphrase_model_path_url[0]
list_paraphrase_model_names = [_x['model_name'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_name = list_paraphrase_model_names[0]
'''
# df_paraphrased_selected_model_full = get_paraphrased_dataframe_selected_models(df_paraphrased_all_model_full, 
#                                                                                       list_paraphrase_model_names)

#df_org_claims_by_scifact= df_org_claims_by_scifact.iloc[:50, :].copy()
df_org_support_major, df_org_refute_major, df_all_cur_model_org_success= get_dataframes_by_majority_org_claim(df_org_claims_by_scifact)
report_dataframes_by_majority_org_claim(df_org_support_major, df_org_refute_major, df_all_cur_model_org_success)

df_all_cur_model_org_success = df_all_cur_model_org_success.drop_duplicates('org_claim', keep='first') 

model_t5_not_fine_tuned = AutoModelForSeq2SeqLM.from_pretrained(paraphrase_model_path_url)
tokenizer_t5_not_fine_tuned = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
_ = model_t5_not_fine_tuned.to(device)
                                                                
df_paraphrased_selected_model_full = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_all_cur_model_org_success, 
                                                                               model_t5 = model_t5_not_fine_tuned, 
                                                                               tokenizer_t5 = tokenizer_t5_not_fine_tuned, 
                                                                               model_name_t5 = paraphrase_model_name)

df_paraphrased_selected_model_full['passed_ner_abr_filter_ic'] = df_paraphrased_selected_model_full.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)


df_paraphrased_selected_model_full[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_selected_model_full.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)


fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'

with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
    pickle.dump(df_paraphrased_selected_model_full, fp)


#df_paraphrased_selected_model_full = pd.read_pickle('../../dfs_generated/paraphrased/paws/cumulative_tech_term_ner_ic/paws_base_no_fine_tune_ParaphraseTargetDirection.org_refute_to_gen_support_0_FT_concat_prev.pkl')


model_t5_not_fine_tuned = model_t5_not_fine_tuned.cpu()    
del model_t5_not_fine_tuned

#df_all_cur_model_org_success = pd.DataFrame(df_all_cur_model_org_success['columns_for_org_claim'].unique(), columns = ['org_claim'])
#split dataframe of support major and refute major
while(True):
    # Get paraphrased sentences with majority of Support or refute.
    df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success= get_dataframes_by_majority_org_claim(df_paraphrased_selected_model_full)
    report_dataframes_by_majority_org_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success)


    # Get successfullt attacked claims after paraphrased
    df_org_support_gen_refute, df_org_refute_gen_support = get_df_succesfully_attacked_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major)
    report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch = CUR_NO_OF_EPOCH_FT)

    report_df_filter(df_org_support_gen_refute, 'org SUP gen REF' ,CUR_NO_OF_EPOCH_FT)
    report_df_filter(df_org_refute_gen_support, 'org REF gen SUP' ,CUR_NO_OF_EPOCH_FT)
    
    
    df_fine_tuning_dataset = None

    ## Select dataset for fine-tuning ::either support major or refute major
    if PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_support_to_gen_refute:
        df_fine_tuning_dataset = df_org_support_gen_refute.copy()
    elif PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_refute_to_gen_support:
        df_fine_tuning_dataset = df_org_refute_gen_support.copy()
    else:
        raise ValueError('Select a direction of fine tuning dataset')

    ##Filter tech terms
    df_fine_tuning_dataset = df_fine_tuning_dataset[df_fine_tuning_dataset['passed_ner_abr_filter_ic'] == True]
    
    df_fine_tuning_dataset = df_fine_tuning_dataset[(df_fine_tuning_dataset['mlnli_label_org_gen'] == 'entailment') &
                                       (df_fine_tuning_dataset['mlnli_label_gen_org'] == 'entailment')]
    
    ## Train model with fine-tuning dataset
    df_fine_tuning_dataset.reset_index(drop=True, inplace=True)
    train_split_size = PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_TRAIN_SPLIT']
    df_train_fine_tune, df_validate_fine_tune = get_train_test_dataset(df_fine_tuning_dataset, train_split_size)

    num_train_epochs = PARAPHRASE_PROJECT_SETTINGS['run_settings']['NUM_OF_EPOCH_REQ_FT']
    fineTuneHyperParam = FineTuneHyperParams(model_name_path = paraphrase_model_path_url, 
                                             num_train_epochs = num_train_epochs, df_train = df_train_fine_tune, 
                                             df_val = df_validate_fine_tune, df_train_val = df_fine_tuning_dataset)

    model_t5_fine_tuned = T5FineTuner(fineTuneHyperParam.args_fine_tune_ns)
    trainer_model_t5_fine_tune = pl.Trainer(**fineTuneHyperParam.train_params_fine_tune)
    trainer_model_t5_fine_tune.fit(model_t5_fine_tuned)

    CUR_NO_OF_EPOCH_FT += 1
    tokenizer_t5 = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
    _ = trainer_model_t5_fine_tune.model.to(device)
    ## Ask fine-tuned model to paraphrase

    ## Evaluate paraphrased dataset
    df_paraphrased_fine_tuned = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_all_cur_model_org_success.iloc[:, :], 
                                              model_t5 = model_t5_fine_tuned, 
                                              tokenizer_t5 = tokenizer_t5, 
                                              model_name_t5 = paraphrase_model_name)

    
    df_paraphrased_fine_tuned['passed_ner_abr_filter_ic'] = df_paraphrased_fine_tuned.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)


    df_paraphrased_fine_tuned[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_fine_tuned.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

    fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'
    #list_paraphrase_model_names[0]+'_'+PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION']+'_'+NUM_OF_EPOCH_REQ_FT
    with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
        pickle.dump(df_paraphrased_fine_tuned, fp)
    
    print('>> before epoch '+str(CUR_NO_OF_EPOCH_FT)+' size was '+str(len(df_paraphrased_selected_model_full)))
    df_paraphrased_selected_model_full = pd.concat([df_paraphrased_selected_model_full, df_paraphrased_fine_tuned], ignore_index=True)
    print('>> after epoch '+str(CUR_NO_OF_EPOCH_FT)+' size was '+str(len(df_paraphrased_selected_model_full)))

In [ ]:
# get all model's no_fine_tuned dataset
#df_paraphrased_all_model_full = get_paraphrased_dataframe_all_model_no_fine_tuned()

#Filter and select dataset only for the selected model

paraphrase_model_path_url = [_x['model_path_or_url'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_path_url = paraphrase_model_path_url[0]
list_paraphrase_model_names = [_x['model_name'] for  _x in PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['list_potential_paraphrase_models'] if _x['is_selected'] == True]
paraphrase_model_name = list_paraphrase_model_names[0]
# df_paraphrased_selected_model_full = get_paraphrased_dataframe_selected_models(df_paraphrased_all_model_full, 
#                                                                                       list_paraphrase_model_names)

#df_org_claims_by_scifact= df_org_claims_by_scifact.iloc[:50, :].copy()
df_org_support_major, df_org_refute_major, df_all_cur_model_org_success= get_dataframes_by_majority_org_claim(df_org_claims_by_scifact)
report_dataframes_by_majority_org_claim(df_org_support_major, df_org_refute_major, df_all_cur_model_org_success)

df_all_cur_model_org_success = df_all_cur_model_org_success.drop_duplicates('org_claim', keep='first') 

model_t5_not_fine_tuned = AutoModelForSeq2SeqLM.from_pretrained(paraphrase_model_path_url)
tokenizer_t5_not_fine_tuned = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
_ = model_t5_not_fine_tuned.to(device)
                                                                
df_paraphrased_selected_model_full = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_all_cur_model_org_success, 
                                                                               model_t5 = model_t5_not_fine_tuned, 
                                                                               tokenizer_t5 = tokenizer_t5_not_fine_tuned, 
                                                                               model_name_t5 = paraphrase_model_name)

df_paraphrased_selected_model_full['passed_ner_abr_filter_ic'] = df_paraphrased_selected_model_full.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)


df_paraphrased_selected_model_full[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_selected_model_full.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)


fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'

with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
    pickle.dump(df_paraphrased_selected_model_full, fp)


#df_paraphrased_selected_model_full = pd.read_pickle('../../dfs_generated/paraphrased/paws/cumulative_tech_term_ner_ic/paws_base_no_fine_tune_ParaphraseTargetDirection.org_refute_to_gen_support_0_FT_concat_prev.pkl')


model_t5_not_fine_tuned = model_t5_not_fine_tuned.cpu()    
del model_t5_not_fine_tuned

#df_all_cur_model_org_success = pd.DataFrame(df_all_cur_model_org_success['columns_for_org_claim'].unique(), columns = ['org_claim'])
#split dataframe of support major and refute major
while(True):
    # Get paraphrased sentences with majority of Support or refute.
    df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success= get_dataframes_by_majority_org_claim(df_paraphrased_selected_model_full)
    report_dataframes_by_majority_org_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success)


    # Get successfullt attacked claims after paraphrased
    df_org_support_gen_refute, df_org_refute_gen_support = get_df_succesfully_attacked_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major)
    report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch = CUR_NO_OF_EPOCH_FT)

    report_df_filter(df_org_support_gen_refute, 'org SUP gen REF' ,CUR_NO_OF_EPOCH_FT)
    report_df_filter(df_org_refute_gen_support, 'org REF gen SUP' ,CUR_NO_OF_EPOCH_FT)
    
    
    df_fine_tuning_dataset = None

    ## Select dataset for fine-tuning ::either support major or refute major
    if PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_support_to_gen_refute:
        df_fine_tuning_dataset = df_org_support_gen_refute.copy()
    elif PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'] == ParaphraseTargetDirection.org_refute_to_gen_support:
        df_fine_tuning_dataset = df_org_refute_gen_support.copy()
    else:
        raise ValueError('Select a direction of fine tuning dataset')

    ##Filter tech terms
    df_fine_tuning_dataset = df_fine_tuning_dataset[df_fine_tuning_dataset['passed_ner_abr_filter_ic'] == True]
    
    df_fine_tuning_dataset = df_fine_tuning_dataset[(df_fine_tuning_dataset['mlnli_label_org_gen'] == 'entailment') &
                                       (df_fine_tuning_dataset['mlnli_label_gen_org'] == 'entailment')]
    
    ## Train model with fine-tuning dataset
    df_fine_tuning_dataset.reset_index(drop=True, inplace=True)
    train_split_size = PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_TRAIN_SPLIT']
    df_train_fine_tune, df_validate_fine_tune = get_train_test_dataset(df_fine_tuning_dataset, train_split_size)

    num_train_epochs = PARAPHRASE_PROJECT_SETTINGS['run_settings']['NUM_OF_EPOCH_REQ_FT']
    fineTuneHyperParam = FineTuneHyperParams(model_name_path = paraphrase_model_path_url, 
                                             num_train_epochs = num_train_epochs, df_train = df_train_fine_tune, 
                                             df_val = df_validate_fine_tune, df_train_val = df_fine_tuning_dataset)

    model_t5_fine_tuned = T5FineTuner(fineTuneHyperParam.args_fine_tune_ns)
    trainer_model_t5_fine_tune = pl.Trainer(**fineTuneHyperParam.train_params_fine_tune)
    trainer_model_t5_fine_tune.fit(model_t5_fine_tuned)

    CUR_NO_OF_EPOCH_FT += 1
    tokenizer_t5 = AutoTokenizer.from_pretrained(paraphrase_model_path_url)  
    _ = trainer_model_t5_fine_tune.model.to(device)
    ## Ask fine-tuned model to paraphrase

    ## Evaluate paraphrased dataset
    df_paraphrased_fine_tuned = get_paraphrased_sentence_with_detail_stat(df_dataset_to_be_paraphrased = df_all_cur_model_org_success.iloc[:, :], 
                                              model_t5 = model_t5_fine_tuned, 
                                              tokenizer_t5 = tokenizer_t5, 
                                              model_name_t5 = paraphrase_model_name)

    
    df_paraphrased_fine_tuned['passed_ner_abr_filter_ic'] = df_paraphrased_fine_tuned.apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)


    df_paraphrased_fine_tuned[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_paraphrased_fine_tuned.apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

    fle_dataframe_to_save = paraphrase_model_name+'_'+str(PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION'])+'_'+str(CUR_NO_OF_EPOCH_FT)+'_FT'
    #list_paraphrase_model_names[0]+'_'+PARAPHRASE_PROJECT_SETTINGS['run_settings']['PARAPHRASE_FT_DATASET_DIRECTION']+'_'+NUM_OF_EPOCH_REQ_FT
    with open('../../dfs_generated/paraphrased/paws/tech_term_ner_mlnli/'+fle_dataframe_to_save+'_concat_prev.pkl', 'wb') as fp:
        pickle.dump(df_paraphrased_fine_tuned, fp)
    
    print('>> before epoch '+str(CUR_NO_OF_EPOCH_FT)+' size was '+str(len(df_paraphrased_selected_model_full)))
    df_paraphrased_selected_model_full = pd.concat([df_paraphrased_selected_model_full, df_paraphrased_fine_tuned], ignore_index=True)
    print('>> after epoch '+str(CUR_NO_OF_EPOCH_FT)+' size was '+str(len(df_paraphrased_selected_model_full)))

  0%|          | 0/340 [00:00<?, ?it/s]

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/home/qudratealahyratu/research/nlp/fact_checking/my_work/scifact/verisci/covid/abstract_retriever.py:53: UserWarning: abstract selector :: returning empty list 
  warnings.warn("abstract selector :: returning empty list ")


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:86: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding`LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch(rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower val

...

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 0: val_loss was not in top 5


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 1: val_loss was not in top 5


  0%|          | 0/340 [00:00<?, ?it/s]

In [62]:
report_dataframes_by_majority_org_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success)

In [63]:
report_df_succesfully_attacked_claim(df_org_support_gen_refute, df_org_refute_gen_support, cur_epoch = CUR_NO_OF_EPOCH_FT)

In [206]:
x = df_org_support_gen_refute.progress_apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)

  0%|          | 0/81 [00:00<?, ?it/s]

In [202]:
#https://stackoverflow.com/questions/29996079/match-a-whole-word-in-a-string-using-dynamic-regex
def filter_and_replace_tech_term_paraphrased_claim(claim_paraphrased, claim_original):
    print('\n>>')
    print(claim_paraphrased)
    print(claim_original)
    #claim_para_trimmed = re.sub('[^a-z]+', ' ', claim_paraphrased.lower())
    df_cur_sentence_word_unq_ner_abr_filtered = df_scispacy_sentence_word_unq_ner_abr_filtered[
        df_scispacy_sentence_word_unq_ner_abr_filtered['claim'] == claim_original
    ]
    for cur_term_row in df_cur_sentence_word_unq_ner_abr_filtered.itertuples(index=False):
        cur_term_row_formatted = r'(?<!\S){}(?!\S)'.format(re.escape(cur_term_row.ner_text))
        res_num = re.findall(cur_term_row_formatted, claim_paraphrased)
        if res_num == []:
            print(cur_term_row_formatted.casefold())
            return False
        
    return True

In [208]:
len(x[x == True])

39

In [181]:
cur_term_row = 'HIV'
claim_paraphrased = 'HIV Having a main partner worsens HIVd outcomes HIV .'
cur_term_row_formatted = r'(?<!\w){}(?!\w)'.format(re.escape(cur_term_row.))
re.findall(cur_term_row_formatted, claim_paraphrased)

['HIV', 'HIV']

In [177]:
x

['HIV', 'HIV', 'HIV']

In [122]:
cur_term_row_formatted

'\\bhiv\\b'

In [101]:
claim_paraphrased.lower().strip()

'having a main partner worsens hiv outcomes.'

In [99]:
re.escape(cur_term_row).lower()

'hiv'

In [ ]:
 filter_and_replace_tech_term_paraphrased_claim

In [70]:
print(df_org_support_gen_refute[df_org_support_gen_refute['mlnli_label_org_gen'] == 'entailment'].shape)
print(df_org_support_gen_refute[
    (df_org_support_gen_refute['mlnli_label_org_gen'] == 'entailment') &
    (df_org_support_gen_refute['mlnli_label_gen_org'] == 'entailment')
].shape)
print(df_org_support_gen_refute[df_org_support_gen_refute['passed_ner_abr_filter_ic'] == True].shape)

(64, 34)
(55, 34)
(4, 34)


In [77]:
df_org_support_gen_refute[df_org_support_gen_refute['passed_ner_abr_filter_ic'] == False][['org_claim', 'gen_claim']].values

array([['A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE).',
        'A cell helper 2 (Th2) environment impedes disease development in patients with systemic Lupus erythematoses'],
       ['ART substantially reduces infectiveness of HIV-positive people.',
        'HIV-positive people receive significantly reduced ART treatment'],
       ['Active caspase-11 protein promotes pyroptosis.',
        'The activating caspase-11 protein promotes pyroptolysis.'],
       ['Adult tissue-resident macrophages are seeded before birth.',
        'Adult tissue-resisting macrophage are germinated before birth.'],
       ['Adult tissue-resident macrophages are seeded before birth.',
        'Adult tissue-based macrophages are seedling before their birth.'],
       ['Angiotensin converting enzyme inhibitors are associated with decreased risk for functional renal insufficiency.',
        'Angiotensin converting enzyme inhibitored are assoc

In [74]:
print(df_org_refute_gen_support[df_org_refute_gen_support['mlnli_label_org_gen'] == 'entailment'].shape)


print(df_org_refute_gen_support[
    (df_org_refute_gen_support['mlnli_label_org_gen'] == 'entailment') &
    (df_org_refute_gen_support['mlnli_label_gen_org'] == 'entailment')
].shape)

print(df_org_refute_gen_support[df_org_refute_gen_support['passed_ner_abr_filter_ic'] == True].shape)

(43, 34)
(36, 34)
(12, 34)


In [85]:
len(df_org_refute_gen_support[df_org_refute_gen_support['mlnli_label_org_gen'] == 'entailment']['org_claim'].unique())

28

In [193]:
'IL-1β'.lower() == 'IL-1β'.casefold()


True

In [1]:
report_df_filter(df_org_support_gen_refute, 'org SUP gen REF' ,CUR_NO_OF_EPOCH_FT)

NameError: name 'report_df_filter' is not defined

## No fine tuned model

In [51]:
import transformers
import pytorch_lightning

In [52]:
print(transformers.__version__)
print(pytorch_lightning.__version__)

4.9.2
1.4.9


In [51]:
df_fine_tuning_dataset

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,...,gen_list_supported_confidence_mean,gen_list_refuted_confidence_mean,gen_comment,common_all,common_support_refute,common_refute_support,common_support_support,common_refute_refute,model,passed_ner_abr_filter_ic
0,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.66,0.0,success,0,0,0,0,0,paws_base_no_fine_tune,True
1,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.71,0.0,success,0,0,0,0,0,paws_base_no_fine_tune,True


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("razent/SciFive-large-Pubmed_PMC")  
model = AutoModelForSeq2SeqLM.from_pretrained("razent/SciFive-large-Pubmed_PMC")


In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
model_t5 = T5ForConditionalGeneration.from_pretrained('Vamsi/T5_Paraphrase_Paws')
tokenizer_t5 = T5Tokenizer.from_pretrained('razent/SciFive-large-Pubmed_PMC')       
_ = model_t5.to(device)

In [ ]:
org_sentence = 'BCL-2 promotes the apoptotic effects of c-Myc.'
#'A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE)'

In [ ]:
text =  "paraphrase: " + org_sentence 

encoding = tokenizer_t5.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


outputs = model_t5.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=50,
    top_p=0.99,
    repetition_penalty=3.5,
    early_stopping=True,
    num_return_sequences=10
)

gen_sentences_t5 = []
for output in outputs:
    line = tokenizer_t5.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    gen_sentences_t5.append(line)

In [ ]:
set(gen_sentences_t5)

In [ ]:
set(gen_sentences_t5)

In [2]:
import re

In [3]:
target_claim = 'BCL-2 promotes the apoptosis effects of c-myce a4pt gene.'
org_term = 'c-Myce A4PT Gene'
term_simple_form = org_term.lower()
#pattern = re.compile(r'\b{}\b'.format(term_simple_form), re.IGNORECASE)
regex_to_search = r'\b(?=\w)' + re.escape(term_simple_form) + r'\b(?!\w)'
pattern = re.compile(r'\b{}\b'.format(regex_to_search), re.IGNORECASE)
s2=pattern.sub(org_term,target_claim)


In [ ]:
pattern

In [4]:
s2

'BCL-2 promotes the apoptosis effects of c-Myce A4PT Gene.'

https://blog.devgenius.io/different-ways-to-replace-occurences-of-a-substring-in-python-strings-2911b1f7bf86

https://betterprogramming.pub/5-ways-to-find-the-index-of-a-substring-in-python-13d5293fc76d

In [ ]:
!conda env list

In [1]:
!/home/qudratealahyratu/anaconda3/envs/scifact/bin/pip install -U spacy[cuda113]

  Using cached spacy-3.2.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached thinc-8.0.13-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (628 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-trf 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.1 which is incompatible.
en-core-web-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.1 which is incompatible.
en-core-web-lg 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.1 which is incompatible.
en-core-sci-sm 0.4.0 requires spacy<3.1.0,>=3.0.1, but you have spacy 3.2.1 which is incompatible.


In [ ]:
import thinc
thinc.__version__

In [44]:
!pip help uninstall


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

Description:
  Uninstall packages.
  
  pip is able to uninstall most installed packages. Known exceptions are:
  
  - Pure distutils packages installed with ``python setup.py install``, which
    leave behind no metadata to determine what files were installed.
  - Script wrappers installed by ``python setup.py develop``.

Uninstall Options:
  -r, --requirement <file>    Uninstall all the packages listed in the given
                              requirements file.  This option can be used
                              multiple times.
  -y, --yes                   Don't ask for confirmation of uninstall
                              deletions.

General Options:
  -h, --help                  Show help.
  --isolated                  Run pip in an isolated mode, ignoring
                              environment variables and user configuration.
  -v, --verbose               Give more 

In [51]:
import pandas as pd
df_paraphrased_selected_model_full = pd.read_pickle('../../dfs_generated/paraphrased/paws/cumulative_tech_term_ner_ic/paws_base_no_fine_tune_ParaphraseTargetDirection.org_refute_to_gen_support_0_FT_concat_prev.pkl')

In [52]:
df_paraphrased_selected_model_full[]

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,...,gen_list_supported_confidence_mean,gen_list_refuted_confidence_mean,gen_comment,common_all,common_support_refute,common_refute_support,common_support_support,common_refute_refute,model,passed_ner_abr_filter_ic
0,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.00,0.58,success,1,0,0,0,1,paws_base_no_fine_tune,True
1,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.00,0.00,no result,0,0,0,0,0,paws_base_no_fine_tune,True
2,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.66,0.00,success,0,0,0,0,0,paws_base_no_fine_tune,True
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.00,0.00,no result,0,0,0,0,0,paws_base_no_fine_tune,True
4,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train,0,1,[],[e8fhlo37.00012],[],[0.6389865875244141],...,0.71,0.00,success,0,0,0,0,0,paws_base_no_fine_tune,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,BCL-2 activation antagonizes the apoptotic eff...,SUPPORTS,[Eliminating BCL-2 yielded rapid loss of leuke...,train,1,2,[tos0t3e8.00028],"[igoq6o3f.00012, i4go7ehv.00025]",[0.5063891410827637],"[0.6773735880851746, 0.612122654914856]",...,0.00,0.53,success,1,0,0,0,1,paws_base_no_fine_tune,False
312,BCL-2 activation antagonizes the apoptotic eff...,SUPPORTS,[Eliminating BCL-2 yielded rapid loss of leuke...,train,1,2,[tos0t3e8.00028],"[igoq6o3f.00012, i4go7ehv.00025]",[0.5063891410827637],"[0.6773735880851746, 0.612122654914856]",...,0.56,0.70,success,3,0,0,1,2,paws_base_no_fine_tune,False
313,BCL-2 activation antagonizes the apoptotic eff...,SUPPORTS,[Eliminating BCL-2 yielded rapid loss of leuke...,train,1,2,[tos0t3e8.00028],"[igoq6o3f.00012, i4go7ehv.00025]",[0.5063891410827637],"[0.6773735880851746, 0.612122654914856]",...,0.75,0.69,success,2,0,1,1,0,paws_base_no_fine_tune,False
314,BCL-2 activation antagonizes the apoptotic eff...,SUPPORTS,[Eliminating BCL-2 yielded rapid loss of leuke...,train,1,2,[tos0t3e8.00028],"[igoq6o3f.00012, i4go7ehv.00025]",[0.5063891410827637],"[0.6773735880851746, 0.612122654914856]",...,0.53,0.51,success,2,0,0,1,1,paws_base_no_fine_tune,False


In [7]:
df_tmp = pd.read_pickle('../../dfs_generated/paraphrased/paws/cumulative_tech_term_ner_ic/paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_0_FT_concat_prev.pkl')

In [11]:
df_tmp.columns

Index(['org_claim', 'ground_label', 'ground_list_rationales', 'source',
       'org_count_support', 'org_count_refute', 'org_list_supported_ids',
       'org_list_refuted_ids', 'org_list_supported_confidence',
       'org_list_refuted_confidence', 'org_list_supported_confidence_mean',
       'org_list_refuted_confidence_mean', 'org_comment', 'gen_claim',
       'gen_count_support', 'gen_count_refute', 'gen_list_supported_ids',
       'gen_list_refuted_ids', 'gen_list_supported_confidence',
       'gen_list_refuted_confidence', 'gen_list_supported_confidence_mean',
       'gen_list_refuted_confidence_mean', 'gen_comment', 'common_all',
       'common_support_refute', 'common_refute_support',
       'common_support_support', 'common_refute_refute', 'model',
       'passed_ner_abr_filter_ic'],
      dtype='object')

In [15]:
df_tmp['passed_ner_abr_filter_ic'].value_counts()

True     3430
False    2584
Name: passed_ner_abr_filter_ic, dtype: int64

In [28]:
df_tmp['passed_ner_abr_filter_ic'] = df_tmp.progress_apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)

  0%|          | 0/6014 [00:00<?, ?it/s]

In [29]:
HTML(df_tmp[df_tmp['passed_ner_abr_filter_ic'] == True][['org_claim', 'gen_claim']].to_html())

,org_claim,gen_claim
0,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,People with severe mental disorder in low and middle income countries receive no treatment.
1,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,In lower and middle-income countries 76-75% of people with severe mental disorder receive no treatment.
2,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with a serious mental disorder receive no treatment in low and middle income countries.
3,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,In low- and middle income countries 76 to 85 percent of people with severe mental disorder receive no treatment
4,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with severe mental disorder get no treatment in low and middle income countries.
6,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with severe mental disorders receive no treatment in low and middle income countries.
7,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with severe mental disorder receive no treatment in low and middle countries
8,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with severe mental disorder receive no treatment in low and moderate income countries
9,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,If only 76-85% of people with severe mental disorder receive no treatment in countries with low and middle incomes.
10,76-85% of people with severe mental disorder receive no treatment in low and middle income countries.,76-85% of people with a severe mental disorder receive no treatment in low and middle income countries.


In [53]:
pd.read_pickle('../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl').columns

Index(['org_claim', 'ground_label', 'ground_list_rationales', 'source',
       'org_count_support', 'org_count_refute', 'org_list_supported_ids',
       'org_list_refuted_ids', 'org_list_supported_confidence',
       'org_list_refuted_confidence', 'org_list_supported_confidence_mean',
       'org_list_refuted_confidence_mean', 'org_comment', 'gen_claim',
       'gen_count_support', 'gen_count_refute', 'gen_list_supported_ids',
       'gen_list_refuted_ids', 'gen_list_supported_confidence',
       'gen_list_refuted_confidence', 'gen_list_supported_confidence_mean',
       'gen_list_refuted_confidence_mean', 'gen_comment', 'common_all',
       'common_support_refute', 'common_refute_support',
       'common_support_support', 'common_refute_refute', 'model',
       'parrot_score'],
      dtype='object')

In [66]:
df_tmp = pd.read_pickle('../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl')[['org_claim', 'ground_label', 'ground_list_rationales', 'source',
       'org_count_support', 'org_count_refute', 'org_list_supported_ids',
       'org_list_refuted_ids', 'org_list_supported_confidence',
       'org_list_refuted_confidence', 'org_list_supported_confidence_mean',
       'org_list_refuted_confidence_mean', 'org_comment']]

In [67]:
df_tmp = df_tmp.drop_duplicates('org_claim', keep='first')
df_tmp.reset_index(drop=True, inplace = True)

In [70]:
df_tmp.to_pickle('../../dfs_generated/scifact/org_claim_ext_roberta_roberta.pkl')